In [2]:
#if needed...

import importnb
import Eva_and_Status as pi

#copy is needed, sorry ^^
import copy

import networkx as nx
#from cdlib import algorithms, readwrite, evaluation, viz, utils

from collections import defaultdict
from collections import OrderedDict
from collections import Counter
from operator import itemgetter
from itertools import groupby
import operator

#import pandas as pd
import numpy
import json
import csv

from os import listdir
import os.path

#import matplotlib.pyplot as plt
#%matplotlib inline

In [3]:
node_labels = sorted([f for f in listdir('TrimesterNode_labels')]) #label directory
projections = sorted([f for f in listdir('Trimester Projections')]) #network directory

#to check if they are the same
print(len(node_labels))
print(len(projections))
print(node_labels)
print(projections)

5
5
['JanApr_17\n.csv', 'MayAug_16\n.csv', 'MayAug_17\n.csv', 'SepDec_16\n.csv', 'SepDec_17\n.csv']
['Proj_JanApr17.csv', 'Proj_MayAug16.csv', 'Proj_MayAug17.csv', 'Proj_SepDec16.csv', 'Proj_SepDec17.csv']


In [4]:
#list of networks
list_g = list()
#for all networks
for i, d in enumerate(projections):
    #build the net
    g = nx.Graph()
    with open('Trimester Projections/' + d,'rt')as f:
        data = csv.reader(f)
        for l in data:
            l = [s.split() for s in l]
            g.add_edge(l[0][0],l[0][1])        
    list_g.append(g) #add to the list

In [5]:
#directory files are ordered, so it works
for i, nl in enumerate(node_labels):
    with open('TrimesterNode_labels/' + nl,'rt')as f:
        data = csv.reader(f)
        for l in data:
            if l[0] in list_g[i].nodes():
                #add the labels to the nodes
                list_g[i].add_node(l[0], labels=l[1])

In [ ]:
#SOME CHECK AND ANALYSIS EXAMPLE

#number of nodes for each net
#for g in list_g:
#    print(g.number_of_nodes())

#labels of the firs net
#print(list_g[0].nodes.data())

In [6]:
for i, g in enumerate(list_g):
    
    #create yourself a path named "Communities_Proj"
    path = "Trimester_Communities_Proj2/" + projections[i]
    #create the directorys in "Communities_Proj
    os.mkdir(path)
    
    #IF ALL NODES ARE LABELED, WE DON'T NEED THIS
    temp_g=list()
    for k, v in g.nodes.data():
        if len(v)!=0:
            temp_g.append(k)
    #new graph without no labeled nodes
    new_g = nx.subgraph(g, temp_g)
    
    # nodename -> map to int
    #use g or new_g
    mapping = dict(zip(new_g, range(0, len(new_g))))
    # reverse function
    inv_map = {v: k for k, v in mapping.items()}
    g_relabel = nx.relabel_nodes(new_g, mapping)
    print(g_relabel.number_of_nodes())
    
    for alpha in [0, 0.7, 0.8, 0.9, 1]:
        #EVA!
        part, com_labels = pi.best_partition(g_relabel, alpha=alpha, beta=1-alpha)
        
        modularity = pi.modularity(part,g_relabel)
        print(modularity)
        purity = pi.purity(com_labels)
        
        #list of list of communities
        sorted_part = sorted(list(part.items()), key=itemgetter(1))
        groups = groupby(sorted_part, key=itemgetter(1))
        group_by_com_attr = [[x[0] for x in v] for k, v in groups]

        #save it with the original namenodes
        original_group = list()
        for el in group_by_com_attr:
            el = copy.deepcopy(el)
            el = nx.subgraph(g_relabel, el)
            el = nx.relabel_nodes(el, inv_map)
            original_group.append(list(el.nodes()))
        
        coms = defaultdict(int)
        for n, c in part.items():
            coms[c] += 1
            
        #print(sorted(dict(coms).items(), key=lambda x: x[0]))
        attr = sorted(com_labels.items(), key=lambda x: x[0])
        
        #the most frequent attr
        max_attrs = list()
        for el in attr:
            max_attrs.append( [el[0], max(el[1].items(), key=operator.itemgetter(1))[0]] )
        
        #save it!
        res = {"alpha": alpha,
           "purity": purity,
           "modularity": modularity,
           "coms_max_label": max_attrs,
           "coms": original_group}
        
        js = json.dumps(res)
        filename = "alpha_" + str(alpha)
        f = open(os.path.join(path, filename), 'w')
        f.write(js)
        f.close()

3517
0.4218679467757036
0.4198547609701002
0.41940580510626646
0.4182347951352903
0.3749060762549112
92
0.40242697698424224
0.36866744084772823
0.40242697698424224
0.3985822809309527
0.3045389388880824
2363
0.37370574253049266
0.37757150218745833
0.3770534465811345
0.37569063277686937
0.32443785702966565
472
0.4825711665576035
0.4809806187518162
0.48265383106847703
0.47714189778335175
0.4313462827384754
2635
0.41866766593152926
0.41949402241113287
0.4200514634121995
0.41809136900738647
0.3595807841693465
